In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#https://github.com/tzamalisp/Human-Activity-Recognition-with-Tensorflow2-and-Keras
from sklearn.model_selection import train_test_split
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Dropout, Input, Conv1D, MaxPooling1D
import tensorflow as tf
import sys
import pandas as pd
import h5py

np.set_printoptions(threshold=sys.maxsize)

path = r'/content/drive/MyDrive/pgan2_colab/RL_acc_wisdm_data.h5'
dataset = 'wisdm'

# - - - get data parameters and split - - - #

hf = h5py.File(path, 'r')
X_source = np.array(hf.get('X'))
y_source = np.array(hf.get('y'))
p_source = np.array(hf.get('p'))
y_onehot_source = np.array(hf.get('y_onehot'))
p_onehot_source = np.array(hf.get('p_onehot'))
print("X source shape: {}".format(X_source.shape))
print("Y source shape: {}".format(y_source.shape))
print("P source shape: {}".format(p_source.shape))

# VARIABLES REGARDING DATA SHAPE, TRAINING

seq_length = X_source.shape[1]
num_channels = X_source.shape[2]
input_shape = (seq_length, num_channels)
num_classes_a = y_onehot_source.shape[1]
num_classes_p = p_onehot_source.shape[1]

# SPLIT INTO TRAINING AND VALIDATION SETS

X_train, X_test, y_train, y_test = train_test_split(X_source, y_source, test_size=0.7, random_state=42)
p_train, p_test, p_train_onehot, p_test_onehot = train_test_split(p_source, p_onehot_source, test_size=0.7, random_state=42)
y_train_onehot, y_test_onehot = train_test_split(y_onehot_source, test_size=0.7, random_state=42)

# SPLIT INTO TRAINING AND VALIDATION SETS
#
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.30, random_state=42)
p_train, p_test, p_train_onehot, p_test_onehot = train_test_split(p_train, p_train_onehot, test_size=0.30, random_state=42)
y_train_onehot, y_test_onehot = train_test_split(y_train_onehot, test_size=0.30, random_state=42)

latent_dim = num_channels #length of random input fed to generator
batch_size_train = y_train.shape[0] #num instances generated for G/D training
batch_size_test = y_test.shape[0]
epoch = 1
vis_freq =100 #100
print(batch_size_train)
print(batch_size_test)
print(y_test_onehot.shape, y_train_onehot.shape, p_test_onehot.shape, p_train_onehot.shape)
#WEIGHTS FOR DIFFERENT TERMS IN THE LOSS FUNCTION

D_p_loss_weight = 1
D_a_loss_weight = 1

a_accuracy = 0.75
p_accuracy = 0.2

# - - - build the model - - - #

# - - - build the model - - - #

def generator(seq_length, latent_dim):
    G_input_shape = (seq_length, latent_dim)
    G_in = Input(shape=G_input_shape)
    G = Dropout(.5)(G_in)
    G = LSTM(128, return_sequences=True, activation="tanh")(G)#uci 128
    G = Dropout(.5)(G)
    G = Dense(latent_dim, activation="tanh")(G)
    G = Model(inputs=G_in, outputs=G)
    G.summary()
    return G

def descriminatorA(input_shape, num_classes):
    
    disc = Sequential(name="D_a")
    
    # Conv1D layer - Input layer --> 01
    disc.add(keras.layers.Conv1D(filters= 64, kernel_size= 3, activation= 'relu', 
                                  input_shape=input_shape, padding= 'valid' , strides= 1))
    
    # Leaky ReLu Layer --> 01
    disc.add(keras.layers.LeakyReLU(alpha=0.1))
    
    # MaxPooling1D layer --> 01
    disc.add(keras.layers.MaxPooling1D(pool_size=2,padding = 'valid'))
    
    # Dropout layer --> 01
    disc.add(keras.layers.Dropout( 0.5))
    
    # Batch Normalization Layer - Applied just efore the activation functions 
    disc.add(keras.layers.BatchNormalization())
    
    # Conv1D layer  --> 02
    disc.add(keras.layers.Conv1D(filters=64, kernel_size=3,activation='relu',padding='valid',strides=1))
    
    # Leaky ReLu Layer --> 01
    disc.add(keras.layers.LeakyReLU(alpha=0.1))
    
    # MaxPooling1D layer --> 01
    disc.add(keras.layers.MaxPooling1D(pool_size= 2,padding= 'valid'))

    # Dropout layer --> 01
    disc.add(keras.layers.Dropout(0.5))

    # Batch Normalization Layer - Applied just efore the activation functions
    disc.add(keras.layers.BatchNormalization())

    # Conv1D layer  --> 03
    disc.add(keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='valid', strides=1))
    
    # Leaky ReLu Layer --> 01
    disc.add(keras.layers.LeakyReLU(alpha=0.1))

    # MaxPooling1D layer --> 01
    disc.add(keras.layers.MaxPooling1D(pool_size=2,padding='valid'))
    
    # Dropout layer --> 01
    disc.add(keras.layers.Dropout(0.5))

    # Conv1D layer  --> 03
    disc.add(keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='valid', strides=1))
    
    # Leaky ReLu Layer --> 01
    disc.add(keras.layers.LeakyReLU(alpha=0.1))

    # MaxPooling1D layer --> 01
    disc.add(keras.layers.MaxPooling1D(pool_size=2,padding='valid'))
    
    # Dropout layer --> 01
    disc.add(keras.layers.Dropout(0.5))

    # Flatten layer --> 01
    disc.add(keras.layers.Flatten())

    # Batch Normalization Layer - Applied just efore the activation functions
    disc.add(keras.layers.BatchNormalization())

    # Dense layer --> 01
    disc.add(keras.layers.Dense(units=100, activation='relu' ))

    # Leaky ReLu Layer --> 01
    disc.add(keras.layers.LeakyReLU(alpha=0.1))

    # Dropout layer --> 01
    disc.add(keras.layers.Dropout(0.5)) 
    
    # Batch Normalization Layer - Applied just efore the activation functions
    disc.add(keras.layers.BatchNormalization())

    # Dense layer - Output layer  --> 02
    disc.add(keras.layers.Dense(num_classes, activation='softmax') )
    
    disc.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    disc.summary()
    return disc

def descriminatorP(input_shape, num_classes):
    classifier = Sequential(name="D_p")
    # Conv1D layer - Input layer --> 01
    classifier.add(keras.layers.Conv1D(filters= 64, kernel_size=3, activation= 'relu', 
                                  input_shape=input_shape, padding= 'valid' , strides= 1))
    
    # Leaky ReLu Layer --> 01
    classifier.add(keras.layers.LeakyReLU(alpha=0.1))
    
    # MaxPooling1D layer --> 01
    classifier.add(keras.layers.MaxPooling1D(pool_size=2,padding = 'valid'))
    
    # Dropout layer --> 01
    classifier.add(keras.layers.Dropout(0.5))
    
    # Batch Normalization Layer - Applied just efore the activation functions 
    classifier.add(keras.layers.BatchNormalization())
    
    # Conv1D layer  --> 02
    classifier.add(keras.layers.Conv1D(filters=64, kernel_size=3,activation='relu',padding='valid',strides=1))
    
    # Leaky ReLu Layer --> 01
    classifier.add(keras.layers.LeakyReLU(alpha=0.1))
    
    # MaxPooling1D layer --> 01
    classifier.add(keras.layers.MaxPooling1D(pool_size= 2,padding= 'valid'))

    # Dropout layer --> 01
    classifier.add(keras.layers.Dropout(0.5))

    # Batch Normalization Layer - Applied just efore the activation functions
    classifier.add(keras.layers.BatchNormalization())

    # Conv1D layer  --> 03
    classifier.add(keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', padding='valid', strides=1))
    
    # Leaky ReLu Layer --> 01
    classifier.add(keras.layers.LeakyReLU(alpha=0.1))

    # MaxPooling1D layer --> 01
    classifier.add(keras.layers.MaxPooling1D(pool_size=2,padding='valid'))
    
    # Dropout layer --> 01
    classifier.add(keras.layers.Dropout(0.5))

    # Flatten layer --> 01
    classifier.add(keras.layers.Flatten())

    # Batch Normalization Layer - Applied just efore the activation functions
    classifier.add(keras.layers.BatchNormalization())

    # Dense layer --> 01
    classifier.add(keras.layers.Dense(units=100, activation='relu' ))

    # Leaky ReLu Layer --> 01
    classifier.add(keras.layers.LeakyReLU(alpha=0.1))

    # Dropout layer --> 01
    classifier.add(keras.layers.Dropout(0.5)) 

    # Batch Normalization Layer - Applied just efore the activation functions
    classifier.add(keras.layers.BatchNormalization())

    # Dense layer - Output layer  --> 02
    classifier.add(keras.layers.Dense(num_classes, activation='softmax') )
    
    classifier.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    classifier.summary()
    return classifier


def create_gan(D_a, D_p, G):
    for layer in D_a.layers:
        layer.trainable = False
    for layer in D_p.layers:
        layer.trainable = False
    GDD = Model(inputs=G.input, outputs=[D_a(G.output), D_p(G.output)])
    GDD.compile(loss={"D_a":"categorical_crossentropy","D_p":"categorical_crossentropy"},
                optimizer="adam", metrics={"D_a":"accuracy",'D_p':"accuracy"},
                loss_weights = { "D_a": D_a_loss_weight,"D_p": D_p_loss_weight})
    GDD.summary()
    return GDD

def create_generated_data(G):
    G_D= Model(inputs= G.input, outputs= G.output)
    G_D.summary()
    return G.output


#FUNCTION FOR TRAINING GENERATOR FROM DBOTH DISCRIMINATOR AND CLASSIFIER OUTPUT

def train_G(batch_size, x, y_onehot, p_onehot, model, latent_dim):

    loss = model.train_on_batch(x, [y_onehot,np.zeros_like(p_onehot)])#`train_on_batch` trains the model and updates the weights.loss = model.train_on_batch(x, [y_onehot, np.zeros_like(p_onehot)])
    return loss

#CREATE GENERATOR AND DISCRIMINATOR

generator_model= generator(seq_length, latent_dim)
discriminator_a_model = descriminatorA(input_shape, num_classes_a)
discriminator_p_model = descriminatorP(input_shape, num_classes_p)
generated_data_model = create_generated_data(generator_model)

#CREATE FULL ARCHITECTURE WHERE OUPUT OF GENERATOR IS FED TO DISCRIMINATOR AND CLASSIFIER



gan = create_gan(discriminator_a_model, discriminator_p_model, generator_model)
test_loss_list = []
train_loss_list = []

#gan.load_weights("/content/drive/MyDrive/colab_Notebook/gan_ckpt_wisdm/variables/variables")
#generator_model.load_weights("/content/drive/MyDrive/colab_Notebook/generator_ckpt_wisdm/variables/variables")
#discriminator_p_model.load_weights("/content/drive/MyDrive/colab_Notebook/discriminator_p_ckpt_wisdm/variables/variables")
#discriminator_a_model.load_weights("/content/drive/MyDrive/colab_Notebook/discriminaror_a_ckpt_wisdm/variables/variables")


while epoch <= 1000000:
    
    train_loss = train_G(batch_size_train, X_train, y_train_onehot, p_train_onehot, gan, latent_dim)
    train_loss_list.append((epoch,train_loss[0],train_loss[1],train_loss[2],train_loss[3],train_loss[4] ))
    print(epoch, 'train', train_loss[3])# gan.metrics_names,
    
    test_loss = gan.test_on_batch(X_test, [y_test_onehot, p_test_onehot])
    test_loss_list.append((epoch,test_loss[0],test_loss[1],test_loss[2],test_loss[3],test_loss[4] ))
    print(epoch, 'test',  test_loss[3])#gan.metrics_names,
    if test_loss[3] > a_accuracy and test_loss[4] <=p_accuracy:
            gen_out = generator_model.predict(X_test)
            
            # save generated data
            
            h5f = h5py.File('/content/drive/MyDrive/pgan2_colab/GN_old'+ dataset+'_data'+str(epoch)+'_a_'+str("{:.2f}".format(test_loss[3]) )+'_p_'+str("{:.2f}".format(test_loss[4]))+'.h5', 'w')
            h5f.create_dataset('X', data=gen_out)
            h5f.create_dataset('y', data=y_test)
            h5f.create_dataset('y_onehot', data=y_test_onehot)
            h5f.create_dataset('p', data=p_test)
            h5f.create_dataset('p_onehot', data=p_test_onehot)
            h5f.close()

            a_accuracy = test_loss[3]
            gan.save("/content/drive/MyDrive/pgan2_colab/oldgan_ckpt_"+dataset)
            discriminator_a_model.save("/content/drive/MyDrive/pgan2_colab/olddiscriminaror_a_ckpt_"+dataset)
            discriminator_p_model.save("/content/drive/MyDrive/pgan2_colab/olddiscriminator_p_ckpt_"+dataset)
            generator_model.save("/content/drive/MyDrive/pgan2_colab/oldgenerator_ckpt_"+dataset)
    
    if epoch % vis_freq == 0:
        pd.DataFrame(train_loss_list).to_csv("/content/drive/MyDrive/pgan2_colab/oldtrain_loss_"+dataset+".csv")
        pd.DataFrame(test_loss_list).to_csv("/content/drive/MyDrive/pgan2_colab/oldtest_loss_"+dataset+".csv")
    
    epoch+=1

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
9579 train 0.3023809492588043
9579 test 0.6007393598556519
9580 train 0.28015872836112976
9580 test 0.5693160891532898
9581 train 0.2825396955013275
9581 test 0.5822550654411316
9582 train 0.27619048953056335
9582 test 0.6062846779823303
9583 train 0.2976190447807312
9583 test 0.6007393598556519
9584 train 0.2698412835597992
9584 test 0.5286506414413452
9585 train 0.2857142984867096
9585 test 0.6007393598556519
9586 train 0.2904762029647827
9586 test 0.5988909602165222
9587 train 0.30000001192092896
9587 test 0.5397412180900574
9588 train 0.27936509251594543
9588 test 0.5009242296218872
9589 train 0.24920634925365448
9589 test 0.5231053829193115
9590 train 0.26746031641960144
9590 test 0.580406665802002
9591 train 0.2769841253757477
9591 test 0.611829936504364
9592 train 0.2666666805744171
9592 test 0.611829936504364
9593 train 0.27222222089767456
9593 test 0.5988909602165222
9594 train 0.2817460298538208
959